In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/IA_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5525777008,IA,"1110 ALBANY PL SE, ORANGE CITY, IA",42.994118,-96.056910,1.916707e+14
1,5928587102,IA,"4600 WESTOWN PKWY, WEST DES MOINES, IA",41.595474,-93.766290,1.915301e+14
2,9406087106,IA,"1312 Locust Street, Suite 202, DES MOINES, IA",41.584756,-93.634255,1.915301e+14
3,7964757003,IA,"5401 VICTORIA AVE, DAVENPORT, IA",41.577026,-90.511300,1.916301e+14
4,4414267000,IA,"500 Walford Rd SW, CEDAR RAPIDS, IA",41.876170,-91.671280,1.911301e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5525777008,IA,"1110 ALBANY PL SE, ORANGE CITY, IA",42.994118,-96.056910,1.916707e+14
1,5928587102,IA,"4600 WESTOWN PKWY, WEST DES MOINES, IA",41.595474,-93.766290,1.915301e+14
2,9406087106,IA,"1312 Locust Street, Suite 202, DES MOINES, IA",41.584756,-93.634255,1.915301e+14
3,7964757003,IA,"5401 VICTORIA AVE, DAVENPORT, IA",41.577026,-90.511300,1.916301e+14
4,4414267000,IA,"500 Walford Rd SW, CEDAR RAPIDS, IA",41.876170,-91.671280,1.911301e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1507,5553777005,IA,"10537 HWY 52 N, DUBUQUE, IA",35.549165,-80.395787,NaN
2009,7197977204,IA,"2100 HIGHWAY 30 West, MOUNT VERNON, IA",42.548330,-114.442696,NaN
2025,9175527000,IA,"309 S. 20TH ST, CLEAR LAKE, IA",45.254760,-92.197655,NaN
2368,4553947003,IA,"3469 Highway 65/69 PO BOX 870, CARLISLE, IA",36.197621,-93.042415,NaN
3195,5847707001,IA,"150 W WASHINGTON ST PO Box 98, NORTH ENGLISH, IA",34.300133,-82.667780,NaN
3230,4107138504,IA,101 E County Line Rd Midland Manufacturing Co ...,34.334760,-82.548805,NaN
3294,3659718407,IA,"3324 Orion Dr, Ames, IA",42.054990,-93.621670,NaN
3559,7398188805,IA,"107 W Main St, Knoxville, IA",41.957237,-77.439880,NaN
3787,4994547007,IA,"203 2nd 2ND ST, DYERSVILLE, IA",30.235006,-93.015110,NaN
3872,1160777202,IA,"12 N 36TH ST, DENISON, IA",34.250347,-82.657660,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('19')] #IA
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/25 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

25


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_15997/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1507,5553777005,IA,"10537 HWY 52 N, DUBUQUE, IA",35.549165,-80.395787,[]
2009,7197977204,IA,"2100 HIGHWAY 30 West, MOUNT VERNON, IA",42.548330,-114.442696,[]
2025,9175527000,IA,"309 S. 20TH ST, CLEAR LAKE, IA",45.254760,-92.197655,[]
2368,4553947003,IA,"3469 Highway 65/69 PO BOX 870, CARLISLE, IA",36.197621,-93.042415,[]
3195,5847707001,IA,"150 W WASHINGTON ST PO Box 98, NORTH ENGLISH, IA",34.300133,-82.667780,[]
3230,4107138504,IA,101 E County Line Rd Midland Manufacturing Co ...,34.334760,-82.548805,[]
3294,3659718407,IA,"3324 Orion Dr, Ames, IA",42.054990,-93.621670,[191690003002005]
3559,7398188805,IA,"107 W Main St, Knoxville, IA",41.957237,-77.439880,[]
3787,4994547007,IA,"203 2nd 2ND ST, DYERSVILLE, IA",30.235006,-93.015110,[]
3872,1160777202,IA,"12 N 36TH ST, DENISON, IA",34.250347,-82.657660,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_15997/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1507,5553777005,IA,"10537 HWY 52 N, DUBUQUE, IA",35.549165,-80.395787,NaN
2009,7197977204,IA,"2100 HIGHWAY 30 West, MOUNT VERNON, IA",42.548330,-114.442696,NaN
2025,9175527000,IA,"309 S. 20TH ST, CLEAR LAKE, IA",45.254760,-92.197655,NaN
2368,4553947003,IA,"3469 Highway 65/69 PO BOX 870, CARLISLE, IA",36.197621,-93.042415,NaN
3195,5847707001,IA,"150 W WASHINGTON ST PO Box 98, NORTH ENGLISH, IA",34.300133,-82.667780,NaN
3230,4107138504,IA,101 E County Line Rd Midland Manufacturing Co ...,34.334760,-82.548805,NaN
3294,3659718407,IA,"3324 Orion Dr, Ames, IA",42.054990,-93.621670,191690003002005
3559,7398188805,IA,"107 W Main St, Knoxville, IA",41.957237,-77.439880,NaN
3787,4994547007,IA,"203 2nd 2ND ST, DYERSVILLE, IA",30.235006,-93.015110,NaN
3872,1160777202,IA,"12 N 36TH ST, DENISON, IA",34.250347,-82.657660,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1507,5553777005,IA,"10537 HWY 52 N, DUBUQUE, IA",35.549165,-80.395787,NaN
2009,7197977204,IA,"2100 HIGHWAY 30 West, MOUNT VERNON, IA",42.548330,-114.442696,NaN
2025,9175527000,IA,"309 S. 20TH ST, CLEAR LAKE, IA",45.254760,-92.197655,NaN
2368,4553947003,IA,"3469 Highway 65/69 PO BOX 870, CARLISLE, IA",36.197621,-93.042415,NaN
3195,5847707001,IA,"150 W WASHINGTON ST PO Box 98, NORTH ENGLISH, IA",34.300133,-82.667780,NaN
3230,4107138504,IA,101 E County Line Rd Midland Manufacturing Co ...,34.334760,-82.548805,NaN
3559,7398188805,IA,"107 W Main St, Knoxville, IA",41.957237,-77.439880,NaN
3787,4994547007,IA,"203 2nd 2ND ST, DYERSVILLE, IA",30.235006,-93.015110,NaN
3872,1160777202,IA,"12 N 36TH ST, DENISON, IA",34.250347,-82.657660,NaN
4580,3648238802,IA,"509 E Maple St, Milton, IA",46.047170,-95.297540,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
1507,5553777005,NaN
2009,7197977204,NaN
2025,9175527000,NaN
2368,4553947003,NaN
3195,5847707001,NaN
3230,4107138504,NaN
3294,3659718407,191690003002005
3559,7398188805,NaN
3787,4994547007,NaN
3872,1160777202,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/IA_fips_scraped.csv") 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,295429,5525777008,IA,"1110 ALBANY PL SE, ORANGE CITY, IA",42.994118,-96.056910,1.916707e+14
1,295430,5928587102,IA,"4600 WESTOWN PKWY, WEST DES MOINES, IA",41.595474,-93.766290,1.915301e+14
2,295431,9406087106,IA,"1312 Locust Street, Suite 202, DES MOINES, IA",41.584756,-93.634255,1.915301e+14
3,295432,7964757003,IA,"5401 VICTORIA AVE, DAVENPORT, IA",41.577026,-90.511300,1.916301e+14
4,295433,4414267000,IA,"500 Walford Rd SW, CEDAR RAPIDS, IA",41.876170,-91.671280,1.911301e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,295429,5525777008,IA,"1110 ALBANY PL SE, ORANGE CITY, IA",42.994118,-96.056910,1.916707e+14,NaN
1,295430,5928587102,IA,"4600 WESTOWN PKWY, WEST DES MOINES, IA",41.595474,-93.766290,1.915301e+14,NaN
2,295431,9406087106,IA,"1312 Locust Street, Suite 202, DES MOINES, IA",41.584756,-93.634255,1.915301e+14,NaN
3,295432,7964757003,IA,"5401 VICTORIA AVE, DAVENPORT, IA",41.577026,-90.511300,1.916301e+14,NaN
4,295433,4414267000,IA,"500 Walford Rd SW, CEDAR RAPIDS, IA",41.876170,-91.671280,1.911301e+14,NaN
...,...,...,...,...,...,...,...,...
8290,303719,7489088500,IA,"3414 Merchant St, Coralville, IA",41.699331,-91.626229,1.910300e+14,NaN
8291,303720,8071677000,IA,"323 E College St PO Box 100, WEST BRANCH, IA",41.673615,-91.343575,1.903145e+14,NaN
8292,303721,8866147009,IA,"2109 BRADY ST, DAVENPORT, IA",41.540848,-90.573975,1.916301e+14,NaN
8293,303722,9846667009,IA,"2403 Muddy Creek Lane, CORALVILLE, IA",41.707798,-91.576940,1.910300e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,295429,5525777008,IA,"1110 ALBANY PL SE, ORANGE CITY, IA",42.994118,-96.056910,1.916707e+14,NaN,191670706011036.0
1,295430,5928587102,IA,"4600 WESTOWN PKWY, WEST DES MOINES, IA",41.595474,-93.766290,1.915301e+14,NaN,191530111141013.0
2,295431,9406087106,IA,"1312 Locust Street, Suite 202, DES MOINES, IA",41.584756,-93.634255,1.915301e+14,NaN,191530051012029.0
3,295432,7964757003,IA,"5401 VICTORIA AVE, DAVENPORT, IA",41.577026,-90.511300,1.916301e+14,NaN,191630129011028.0
4,295433,4414267000,IA,"500 Walford Rd SW, CEDAR RAPIDS, IA",41.876170,-91.671280,1.911301e+14,NaN,191130107002070.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5525777008,IA,"1110 ALBANY PL SE, ORANGE CITY, IA",42.994118,-96.056910,191670706011036.0
1,5928587102,IA,"4600 WESTOWN PKWY, WEST DES MOINES, IA",41.595474,-93.766290,191530111141013.0
2,9406087106,IA,"1312 Locust Street, Suite 202, DES MOINES, IA",41.584756,-93.634255,191530051012029.0
3,7964757003,IA,"5401 VICTORIA AVE, DAVENPORT, IA",41.577026,-90.511300,191630129011028.0
4,4414267000,IA,"500 Walford Rd SW, CEDAR RAPIDS, IA",41.876170,-91.671280,191130107002070.0
...,...,...,...,...,...,...
8290,7489088500,IA,"3414 Merchant St, Coralville, IA",41.699331,-91.626229,191030003031003.0
8291,8071677000,IA,"323 E College St PO Box 100, WEST BRANCH, IA",41.673615,-91.343575,190314505003011.0
8292,8866147009,IA,"2109 BRADY ST, DAVENPORT, IA",41.540848,-90.573975,191630116003050.0
8293,9846667009,IA,"2403 Muddy Creek Lane, CORALVILLE, IA",41.707798,-91.576940,191030003061000.0


In [21]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()]

LoanNumber       24
BorrowerState    24
full_add         24
Lat              24
Long             24
FIPS_z            0
dtype: int64

Convert to File

In [19]:
fips_IA = fips_merge_drop

In [20]:
fips_IA.to_csv("../../../data/state_data/geo/geo_fips/23/IA_fips_scraped.csv") 